In [12]:
pip install pdfminer.six pandas openpyxl


In [5]:
pip install --upgrade pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 75.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [1]:
pip install pymupdf pdfplumber pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.2 MB/s eta 0:00:00


In [10]:
import fitz  # PyMuPDF

pdf_path = "test3 (1).pdf"
doc = fitz.open(pdf_path)

for page in doc:
    text = page.get_text("text")
    print(text)  # Check the extracted text structure


BANK NAME    : PUNJAB AND SIND BANK                                  26-06-2023
BRANCH NAME  : JALANDHAR SME                                          13:15:47
ADDRESS      : 201 INDUSTRIAL AREA
               JALANDHAR
CITY         : JALANDHAR (PUNJAB)
PIN CODE     : 144001
STATE        : Punjab        INDIA                                   Page No: 1
IFSC Code    : PSIB0000697
MICR Code    : 144023019                                             Phone no: 
0181-5018936
REPORT PRINTED BY : MANPRIT KAUR         Sol id :  0697
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
------------------------------
Account No   :   06971300000866                       Nomination Registered   : 
No
A/C Name     :   M/S. KAMAL BRICKS INDUSTRY           Nominee Name            :
Address      :   560 MOTA SINGH NAGAR
                 JALANDHAR
City         :   JALANDHAR (PPin Code  : 144001

In [13]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextBoxHorizontal, LTTextLineHorizontal
import pandas as pd

pdf_path = "test3 (1).pdf"  # Path to your PDF
output_excel = "extracted_tables_pdfminer.xlsx"

# List to store table rows
table_data = []

# Extract text elements with positions
for page_layout in extract_pages(pdf_path):
    page_text = []
    for element in page_layout:
        if isinstance(element, (LTTextBoxHorizontal, LTTextLineHorizontal)):
            x0, y0, x1, y1 = element.bbox  # Get bounding box
            text = element.get_text().strip()
            page_text.append((x0, y0, text))  # Store position and text

    # Sort words by Y-coordinates (rows) first, then X-coordinates
    page_text.sort(key=lambda item: (-item[1], item[0]))

    # Group words into table-like structure
    table_row = []
    last_y = None
    for x, y, text in page_text:
        if last_y is None or abs(last_y - y) > 10:  # New row if Y difference is large
            if table_row:
                table_data.append(table_row)
            table_row = []
        table_row.append(text)
        last_y = y

    if table_row:
        table_data.append(table_row)  # Add last row

# Convert table to DataFrame
df = pd.DataFrame(table_data)

# Save to Excel
df.to_excel(output_excel, index=False, header=False)

print(f"Tables extracted and saved to {output_excel}")


Tables extracted and saved to extracted_tables_pdfminer.xlsx


In [11]:
import pdfplumber

with pdfplumber.open("test3 (1).pdf") as pdf:
    for i, page in enumerate(pdf.pages):
        table = page.extract_table()
        if table:
            print(f"Table found on Page {i+1}:")
            for row in table:
                print(row)


In [6]:
import pandas as pd

tables = []  # Store tables

with pdfplumber.open("test (1).pdf") as pdf:
    for i, page in enumerate(pdf.pages):
        table = page.extract_table()
        if table:
            df = pd.DataFrame(table)
            tables.append(df)

# Save to Excel
with pd.ExcelWriter("output.xlsx") as writer:
    for idx, df in enumerate(tables):
        df.to_excel(writer, sheet_name=f"Table_{idx+1}", index=False)


IndexError: At least one sheet must be visible

In [14]:
import pandas as pd

# Load the extracted Excel file
input_file = "extracted_tables_pdfminer.xlsx"  # Change if filename is different
output_file = "formatted_transactions.xlsx"  # New cleaned file

# Read the first sheet (if multiple, specify sheet_name)
df = pd.read_excel(input_file, sheet_name=0, header=None)

# Define expected column headers
headers = ["Date", "Transaction Type", "Details", "Debit", "Credit", "Balance"]

# Data Cleaning & Transformation
cleaned_data = []
for index, row in df.iterrows():
    row_values = row.dropna().tolist()  # Remove NaN values

    # Basic check: Ensure the row has enough columns
    if len(row_values) >= len(headers):
        cleaned_data.append(row_values[:len(headers)])  # Trim extra columns if any

# Convert to DataFrame with headers
df_cleaned = pd.DataFrame(cleaned_data, columns=headers)

# Convert Date column to datetime format (if applicable)
df_cleaned["Date"] = pd.to_datetime(df_cleaned["Date"], errors="coerce")

# Save the cleaned table
df_cleaned.to_excel(output_file, index=False)

print(f"Formatted data saved to {output_file}")


Formatted data saved to formatted_transactions.xlsx


In [15]:
import re

# Define a function to extract transactions from raw data
def extract_transactions(df):
    structured_data = []

    for row in df[0]:  # Iterate through the single-column extracted text
        match = re.match(r"(\d{2}-[A-Z]{3}-\d{4})\s+([TC])\s+(.*?)\s+([\d,]+\.?\d*)\s+([\d,]+\.?\d*)(Dr|Cr)?", row)
        if match:
            date, txn_type, description, amount, balance, dr_cr = match.groups()
            structured_data.append([date, txn_type, description, amount, balance, dr_cr])

    return structured_data

# Apply extraction function
transactions = extract_transactions(df)

# Convert to structured DataFrame
structured_df = pd.DataFrame(transactions, columns=["Date", "Type", "Description", "Amount", "Balance", "Dr/Cr"])

# Show first few rows of structured data
structured_df.head()


,Date,Type,Description,Amount,Balance,Dr/Cr


In [16]:
df.head(30)

,0
0,BANK NAME : PUNJAB AND SIND BANK ...
1,Open Date : 31-05-2017
2,Statement of account for the period of 01...
3,"15,000.00 29,64,292.66Dr\n26-Jul-2022 C ..."
4,BANK NAME : PUNJAB AND SIND BANK ...
5,Open Date : 31-05-2017
6,Statement of account for the period of 01...
7,"33,255.00 30,29,311.06Dr\n..."
8,BANK NAME : PUNJAB AND SIND BANK ...
9,Open Date : 31-05-2017


In [17]:
# Step 1: Split multi-line rows into individual lines
split_rows = df[0].str.split("\n").explode().reset_index(drop=True)

# Step 2: Define refined regex pattern for transactions
transaction_pattern = re.compile(
    r"(\d{2}-[A-Z]{3}-\d{4})\s+([TC])\s+(.*?)\s+([\d,]+\.?\d*)\s+([\d,]+\.?\d*)(Dr|Cr)?"
)

# Step 3: Extract transactions
structured_data = []

for row in split_rows:
    match = transaction_pattern.search(row)
    if match:
        date, txn_type, description, amount, balance, dr_cr = match.groups()
        structured_data.append([date, txn_type, description.strip(), amount, balance, dr_cr])

# Step 4: Convert to DataFrame
structured_df = pd.DataFrame(structured_data, columns=["Date", "Type", "Description", "Amount", "Balance", "Dr/Cr"])

# Show the first few rows
structured_df.head()


,Date,Type,Description,Amount,Balance,Dr/Cr


In [18]:
split_rows.head(30)

,0
0,BANK NAME : PUNJAB AND SIND BANK ...
1,BRANCH NAME : JALANDHAR SME ...
2,ADDRESS : 201 INDUSTRIAL AREA
3,JALANDHAR
4,CITY : JALANDHAR (PUNJAB)
5,PIN CODE : 144001
6,STATE : Punjab INDIA ...
7,IFSC Code : PSIB0000697
8,MICR Code : 144023019 ...
9,0181-5018936


In [19]:
# Step 1: Identify transaction-like rows (starting with a date pattern)
import numpy as np

def is_transaction(row):
    """Checks if a row starts with a transaction date pattern (DD-MMM-YYYY)."""
    return bool(re.match(r"\d{2}-[A-Z]{3}-\d{4}", row.strip()))

# Step 2: Merge multi-line transactions
merged_rows = []
temp_row = ""

for row in split_rows:
    if is_transaction(row):  # New transaction starts
        if temp_row:  # Save previous transaction
            merged_rows.append(temp_row.strip())
        temp_row = row  # Start new transaction
    else:
        temp_row += " " + row  # Append to current transaction

if temp_row:  # Append the last transaction
    merged_rows.append(temp_row.strip())

# Step 3: Define improved regex pattern to extract structured transaction details
transaction_pattern = re.compile(
    r"(\d{2}-[A-Z]{3}-\d{4})\s+([TC])\s+(.*?)\s+([\d,]+\.?\d*)\s+([\d,]+\.?\d*)(Dr|Cr)?"
)

# Step 4: Extract transactions into structured format
structured_data = []

for row in merged_rows:
    match = transaction_pattern.search(row)
    if match:
        date, txn_type, description, amount, balance, dr_cr = match.groups()
        structured_data.append([date, txn_type, description.strip(), amount, balance, dr_cr])

# Step 5: Convert structured data to DataFrame
structured_df = pd.DataFrame(structured_data, columns=["Date", "Type", "Description", "Amount", "Balance", "Dr/Cr"])

# Step 6: Convert numeric values to proper format
structured_df["Amount"] = structured_df["Amount"].str.replace(",", "").astype(float)
structured_df["Balance"] = structured_df["Balance"].str.replace(",", "").astype(float)

# Show first few rows of cleaned structured data
structured_df.head()


,Date,Type,Description,Amount,Balance,Dr/Cr


In [20]:
merged_rows[:20]

['BANK NAME    : PUNJAB AND SIND BANK                                  26-06-2023 BRANCH NAME  : JALANDHAR SME                                          13:15:47 ADDRESS      : 201 INDUSTRIAL AREA                JALANDHAR CITY         : JALANDHAR (PUNJAB) PIN CODE     : 144001 STATE        : Punjab        INDIA                                   Page No: 1 IFSC Code    : PSIB0000697 MICR Code    : 144023019                                             Phone no:  0181-5018936 REPORT PRINTED BY : MANPRIT KAUR         Sol id :  0697 -------------------------------------------------------------------------------- -------------------------------------------------------------------------------- ------------------------------ Account No   :   06971300000866                       Nomination Registered   :  No A/C Name     :   M/S. KAMAL BRICKS INDUSTRY           Nominee Name            : Address      :   560 MOTA SINGH NAGAR                  JALANDHAR City         :   JALANDHAR (PPin Code  : 1440